# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [2]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [5]:
# URL de la page avec les fichiers PDF
url = "https://max.de.wilde.web.ulb.be/camille/"

# Crée un dossier pour les fichiers PDF s'il n'existe pas déjà
pdf_dir = "tp1_pdfs"
if not os.path.exists(pdf_dir):
    os.makedirs(pdf_dir)

# Récupérer le contenu de la page
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Trouver tous les liens qui mènent à des fichiers PDF
pdf_links = soup.find_all('a', href=re.compile(r".*\.pdf$"))

# Télécharger chaque fichier PDF
for link in pdf_links:
    pdf_url = link.get('href')
    
    # S'assurer que le lien est complet
    if not pdf_url.startswith("http"):
        pdf_url = url + pdf_url

    # Nom du fichier PDF
    pdf_name = os.path.join(pdf_dir, pdf_url.split('/')[-1])
    
    print(f"Téléchargement de {pdf_name}...")
    
    # Télécharger le fichier PDF
    try:
        with requests.get(pdf_url, stream=True) as r:
            r.raise_for_status()
            with open(pdf_name, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"{pdf_name} téléchargé avec succès.")
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement de {pdf_name} : {e}")

Téléchargement de tp1_pdfs/KB_JB230_1892-08-07_01-0003.pdf...
tp1_pdfs/KB_JB230_1892-08-07_01-0003.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB427_1920-01-10_01-00004.pdf...
tp1_pdfs/KB_JB427_1920-01-10_01-00004.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB555_1836-02-08_01-00002.pdf...
tp1_pdfs/KB_JB555_1836-02-08_01-00002.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB638_1860-05-21_01-00002.pdf...
tp1_pdfs/KB_JB638_1860-05-21_01-00002.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB773_1918-11-30_01-00002.pdf...
tp1_pdfs/KB_JB773_1918-11-30_01-00002.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB838_1887-12-28_01-00003.pdf...
tp1_pdfs/KB_JB838_1887-12-28_01-00003.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB230_1903-10-16_01-0002.pdf...
tp1_pdfs/KB_JB230_1903-10-16_01-0002.pdf téléchargé avec succès.
Téléchargement de tp1_pdfs/KB_JB427_1933-01-04_01-00003.pdf...
tp1_pdfs/KB_JB427_1933-01-04_01-00003.

In [4]:
# Affichage du nombre d'articles récupérés
len(articles)

0

In [ ]:
# Affichage des 10 premières entrées
articles[:10]

## Création d'un dataframe avec les liens et les titres des articles


In [ ]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

In [ ]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [ ]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

### Nettoyage du texte à l'aide d'expressions régulières

In [ ]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp